 Sequential Keras Model 
 <hr>
<code>
from tf.keras.models import Sequential
from tf.keras.layers import Conv2, MaxPooling2D, Flatten, Dense
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(10, activation='softmax')
])
</code>

Functional API
<hr>
<code>  
from tf.keras.models import Model
from tf.keras.layers import Input, Conv2, MaxPooling2D, Flatten, Dense
inputs = Input(shape=(32, 32, 3))
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=x)
</code>

horizontal line
<hr> 
<ol><li>1121232</li></ol>

<code>
    model.fit(train_dataset,
          epochs=60,
          validation_data=test_dataset,
          validation_freq=1)
</code>

### Setting up a data pipeline
For each model we define in Keras, we need a data pipeline to feed it labeled training data. A data pipeline performs the following tasks:

* **Loading:** Copying the dataset (e.g. images and labels) from storage into the program's memory.
* __Preprocessing:__ transforming the dataset. For example, in image classification, we might resize, whiten, shuffle, or batch images.
* <b>Feeding:</b> shoveling examples from a dataset into a training loop.

#### Loading data from storage
First, we load CIFAR-10 from storage into numpy ndarrays:

In [2]:
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler


In [10]:
#simple configs for data pipeline

NUM_GPUS = 2
BS_PER_GPU = 128
NUM_EPOCHS = 60

HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
NUM_CLASSES = 10
NUM_TRAIN_SAMPLES = 50000

BASE_LEARNING_RATE = 0.1
LR_SCHEDULE = [(0.1, 30), (0.01, 45)]

In [3]:
(x, y), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
#debug dataset
print(type(x), type(y))
print(x.shape, y.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(50000, 32, 32, 3) (50000, 1)


After this point we can directly feed this data into training. However, to get more accurate model we need to preprocess the data before feeding into the model. To do so, we will build data preprocessing pipeline to achieve this. We will be applying **tf.Data** class.

### The tf.data.Dataset class
The TensorFlow Dataset class serves two main purposes:

* It acts as a container that holds training data.
* It can be used to perform alterations on elements of the training data.

We instantiate a tensorflow.data.Dataset object for the CIFAR-10 dataset as follows:

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9153819625526386467
]


In [5]:
# Instantiate the Dataset class.
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))

During training, the CIFAR-10 training examples stored in **train_dataset** will be accessed via the **take()** iterator:

```
for image, label in train_dataset.take(1):
    (image.shape, label.shape)
```

In [7]:
for image, label in train_dataset.take(1):
    print(image.shape, label.shape)

(32, 32, 3) (1,)


### Preprocessing steps to apply

1. Data augmentation
Augmentation is often used to "inflate" training datasets, which can improve generalization performance.

Let's augment the CIFAR-10 dataset by performing the following steps on every image:

1. Pad the image with a black, four-pixel border.
2. Randomly crop a 32 x 32 region from the padded image.
3. Flip a coin to determine if the image should be horizontally flipped.

We achieve this by first defining a function that, given an image, performs the Steps 1-3 above:

In [8]:
def augmentation(x, y):
    x = tf.image.resize_with_crop_or_pad(
        x, HEIGHT + 8, WIDTH + 8)
    x = tf.image.random_crop(x, [HEIGHT, WIDTH, NUM_CHANNELS])
    x = tf.image.random_flip_left_right(x)
    return x, y

Next, we call the method **map**.

This call returns a new Dataset object that contains the result of passing each image in CIFAR-10 into augmentation. This new object will emit transformed images in the original order:

In [11]:
train_dataset = train_dataset.map(augmentation)

2. Shuffling

We randomly shuffle the dataset. ***TensorFlow Dataset has a shuffle method***, which can be chained to our augmentation as follows:

In [12]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000))

3. Normalization

It's common practice to normalize data. Here, define a function that linearly scales each image to have zero mean and unit variance:



In [14]:
def normalize(x, y):
  x = tf.image.per_image_standardization(x)
  return x, y

Next, we chain it with our augmentation and shuffling operations:

In [15]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000)
                 .map(normalize))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


4. Batching

Finally, we batch the dataset. We set drop_remainder to True to remove enough training examples so that the training set's size is divisible by batch_size.

In [17]:
#FULL PIPELINE
train_dataset = (train_dataset.map(augmentation)
                 .map(normalize)
                 .shuffle(50000)
                 .batch(128, drop_remainder=True))

## Training the model
A Keras model needs to be compiled before training. Compilation essentially defines three things: the loss function, the optimizer and the metrics for evaluation:


```
model.compile(
          loss='sparse_categorical_crossentropy',
          optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
          metrics=['accuracy'])
 ```

Keras uses the fit API to train a model.

```
model.fit(train_dataset,
          epochs=60,
          validation_data=test_dataset,
          validation_freq=1)

```

To evaluate the model, call the evaluate method with the test dataset

```
model.evaluate(test_loader)

```

## Adding callbacks
Often we need to perform custom operations during training. For example, you might want to log statistics during the training for debugging or optimization purposes; implement a learning rate schedule to improve the efficiency of training; or save visual snapshots of filter banks as they converge. In TensorFlow 2, you can use the callback feature to implement customized events during training.

Tensorboard

TensorBoard is mainly used to log and visualize information during training. It is handy for examining the performance of the model. Tensorboard support is provided via the ***tensorflow.keras.callbacks.TensorBoard*** callback function:

```
from tensorflow.keras.callbacks import TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(
    log_dir=log_dir, update_freq='batch', histogram_freq=1)
  
model.fit(...,
          callbacks=[tensorboard_callback])
```


### Learning Rate Schedule

Often, we would like to have fine control of learning rate as the training progresses. A custom learning rate schedule can be implemented as callback functions. Here, we create a customized schedule function that decreases the learning rate using a step function (at 30th epoch and 45th epoch). This schedule is converted to a ***keras.callbacks.LearningRateScheduler*** and attached to the **fit** function.

In [3]:
BASE_LEARNING_RATE = 0.1
LR_SCHEDULE = [(0.1, 30), (0.01, 45)]
BS_PER_GPU = 128
def schedule(epoch):
  initial_learning_rate = BASE_LEARNING_RATE * BS_PER_GPU / 128
  learning_rate = initial_learning_rate
  for mult, start_epoch in LR_SCHEDULE:
    if epoch >= start_epoch:
      learning_rate = initial_learning_rate * mult
    else:
      break
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

```
from tensorflow.keras.callbacks import LearningRateScheduler

lr_schedule_callback = LearningRateScheduler(schedule)

model.fit(...,
          callbacks=[..., lr_schedule_callback])

```

### Summary
This tutorial explains the basic of TensorFlow 2.0 with image classification as an example. The important things to remember are:

* Data pipeline with TensorFlow 2's dataset API
* Train, evaluation, save and restore models with Keras (TensorFlow 2's official high-level API)
* Customized training with callbacks